In [1]:
import re
import sys
import json


import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [2]:
from nltk.lm        import Vocabulary
# from nltk.lm.models import MLE
from nltk.lm.models import KneserNeyInterpolated
from nltk.util      import ngrams

In [3]:
# path = "../hand_labeled/"
# datalist = ['DCM', 'DIT', 'IRS']
# error_types = ['Unclear intention', 'Wrong information',
#  'Ignore question', 'Topic transition error', 
#  'Lack of information', 'Repetition', 
#  'Contradiction', 'Self-contradiction',
#   'Lack of common sense', 'Semantic error',
#    'Grammatical error', 'Ignore proposal', 
#    'Ignore offer', 'Lack of sociality', 
#    'Uninterpretable', 'Ignore greeting', 
#    'No-Err']

In [4]:
# convs = read_conv(path, datalist)

In [5]:
# path = "../hand_labeled/"
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

error = "Grammatical error"
errors = ['Grammatical error', "Uninterpretable"]
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            if not ut.utt[-1] in ["？", "！", "。", "!"]:
                sys_utt.append( clean_text( ut.utt+"。" ))
                # sys_utt.append(ut.utt)
            else:   
                sys_utt.append(ut.utt)
            if ut.is_error_included(errors):
                y.append(1)
            else:
                y.append(0)

In [6]:
print("len of y:{0}, error '{1}' counts:{2}".format(len(y), error, y.count(1)))

len of y:1386, error 'Grammatical error' counts:7


In [7]:
modelM = DataManager("../models/utterance/")
# model_name = "KLM_nucc.pickle"
# model_name = "KLM_phrase.pickle"
n = 3
model_name = "KLM_phrase_n={0}.pickle".format(n)
# model_name = "KLM_phrase_n={0}_noun.pickle".format(n)
model_name = "KLM_phrase_nucc_n={0}.pickle".format(n)
model_name = "KLM_phrase_nucc_n={0}_noun2.pickle".format(n)
# model_name = "KLM_phrase_nucc_n={0}_orth.pickle".format(n)
lm = modelM.load_data(model_name)

success load : ../models/utterance/KLM_phrase_nucc_n=3_noun2.pickle


In [8]:
import math
import pprint


def sentence2score(sentence, lm, N):
    # filled = fill_SYMBOL( sentence2normalize_independent(sentence) )
    filled = fill_SYMBOL( sentence2normalize_noun(sentence) )
    # filled = fill_SYMBOL( sentence2morpheme(sentence) )
    filled_pos = fill_SYMBOL( sentence2pos(sentence) )
    # print(filled)
    # print(filled_pos)
    ngram_text = []
    ngram_pos = []
    for L, P in zip(filled,filled_pos):
        for i in range(len(L)-N+1):
            # print(L[i:i+N])
            ngram_text.append(L[i:i+N])
            ngram_pos.append(P[i:i+N])
    # pprint.pprint(ngram_text)
    all_score = 0
    function_score = 0
    # デフォルトで1
    function_num = 1

    under = 1 / (10*len( lm.vocab.counts ) )

    for ngram, pgram in zip(ngram_text, ngram_pos):
        context = (ngram[:-1])
        context_pos = pgram[:-1]
        # print(context)
        # for word in lm.context_counts(lm.vocab.lookup(context)): # 文脈に続く単語一覧の取得
            
        score = lm.score(ngram[-1], context) + under
        log_score = math.log2(score)
        # print("context : {0}|{1} ->".format(context, ngram[-1:]), log_score)

        if "助動詞" in context_pos[1] or "助詞" in context_pos[1] or "助動詞" in context_pos[0] or "助詞" in context_pos[0]:
            # print("\tcontext : {0}| ->".format(context), log_score)
            function_score += log_score
            function_num += 1

        all_score += log_score
    # print(all_score/len(ngram_text))
    return all_score/len(ngram_text)
    # return function_score/function_num

In [9]:
def sentence2score_print(sentence, lm, N):
    # filled = fill_SYMBOL( sentence2normalize_independent(sentence) )
    filled = fill_SYMBOL( sentence2normalize_noun(sentence) )
    # filled = fill_SYMBOL( sentence2morpheme(sentence) )
    filled_pos = fill_SYMBOL( sentence2pos(sentence) )
    # print(filled)
    # print(filled_pos)
    ngram_text = []
    ngram_pos = []
    for L, P in zip(filled,filled_pos):
        for i in range(len(L)-N+1):
            # print(L[i:i+N])
            ngram_text.append(L[i:i+N])
            ngram_pos.append(P[i:i+N])
    # pprint.pprint(ngram_text)
    all_score = 0
    function_score = 0
    # デフォルトで1
    function_num = 1
    
    # perple
    # return lm.perplexity(ngram_text)/len(ngram_text)

    under = 1 / (10*len( lm.vocab.counts ) )

    for ngram, pgram in zip(ngram_text, ngram_pos):
        context = (ngram[:-1])
        context_pos = pgram[:-1]
        # print(context)
        # for word in lm.context_counts(lm.vocab.lookup(context)): # 文脈に続く単語一覧の取得
            
        score = lm.score(ngram[-1], context) + under
        log_score = math.log2(score)
        print("context : {0}|{1} ->".format(context, ngram[-1:]), log_score)

        if "助動詞" in context_pos[1] or "助詞" in context_pos[1] or "助動詞" in context_pos[0] or "助詞" in context_pos[0]:
            print("\tcontext : {0}| ->".format(context), log_score)
            function_score += log_score
            function_num += 1

        all_score += log_score
    # print(all_score/len(ngram_text))
    return all_score/len(ngram_text)
    # return function_score/function_num

In [10]:
1 / (10*len( lm.vocab.counts ) )

7.383343177790904e-06

In [11]:

sentence = "最近とても暑いですから。"
sentence = "魚はおいしいんですか？？海は素晴らしいですね。"
# sentence = "魚はおいんですか？？海は素晴らしいですね。"
sentence2score_print(sentence, lm, N=n)

context : ['FOS', 'FOS']|['名詞-普通名詞-一般'] -> -2.2410860748936408
context : ['FOS', '名詞-普通名詞-一般']|['は'] -> -3.4343199954087154
context : ['名詞-普通名詞-一般', 'は']|['おいしい'] -> -9.838764726062092
	context : ['名詞-普通名詞-一般', 'は']| -> -9.838764726062092
context : ['は', 'おいしい']|['ん'] -> -5.360841159571571
	context : ['は', 'おいしい']| -> -5.360841159571571
context : ['おいしい', 'ん']|['です'] -> -2.2660141866750627
	context : ['おいしい', 'ん']| -> -2.2660141866750627
context : ['ん', 'です']|['か'] -> -2.1426234312313257
	context : ['ん', 'です']| -> -2.1426234312313257
context : ['です', 'か']|['？'] -> -17.044164982534674
	context : ['です', 'か']| -> -17.044164982534674
context : ['か', '？']|['？'] -> -13.587959565815042
	context : ['か', '？']| -> -13.587959565815042
context : ['？', '？']|['EOS'] -> -13.587959565815042
context : ['？', 'EOS']|['EOS'] -> -13.587959565815042
context : ['FOS', 'FOS']|['名詞-普通名詞-一般'] -> -2.2410860748936408
context : ['FOS', '名詞-普通名詞-一般']|['は'] -> -3.4343199954087154
context : ['名詞-普通名詞-一般', 'は']|['素晴らし

-5.640032832597145

In [12]:
error_samples = [['ご存知ですいます。'],
 ['魚はおいんですか？？海は素晴らしいですね。'],
 ['もちろんですってのが元気ですかにいくないですよ。'],
 ['熱中症に気をつけか？？']]

for s in error_samples:
    print("{0}:{1}".format(s[0], sentence2score_print(s[0], lm, N=n)))

context : ['FOS', 'FOS']|['名詞-普通名詞-一般'] -> -2.2410860748936408
context : ['FOS', '名詞-普通名詞-一般']|['です'] -> -5.660978421858904
context : ['名詞-普通名詞-一般', 'です']|['い'] -> -13.261981394824302
	context : ['名詞-普通名詞-一般', 'です']| -> -13.261981394824302
context : ['です', 'い']|['ます'] -> -0.15197942359614935
	context : ['です', 'い']| -> -0.15197942359614935
context : ['い', 'ます']|['。'] -> -0.6692450487067928
	context : ['い', 'ます']| -> -0.6692450487067928
context : ['ます', '。']|['EOS'] -> -0.0008155660583266521
	context : ['ます', '。']| -> -0.0008155660583266521
context : ['。', 'EOS']|['EOS'] -> 1.0041833985722283e-05
ご存知ですいます。:-3.1408679840148754
context : ['FOS', 'FOS']|['名詞-普通名詞-一般'] -> -2.2410860748936408
context : ['FOS', '名詞-普通名詞-一般']|['は'] -> -3.4343199954087154
context : ['名詞-普通名詞-一般', 'は']|['おい'] -> -14.271474764266397
	context : ['名詞-普通名詞-一般', 'は']| -> -14.271474764266397
context : ['は', 'おい']|['ん'] -> -0.15197942359614935
	context : ['は', 'おい']| -> -0.15197942359614935
context : ['おい', 'ん']|['です'] 

- N=4 gram の全品詞，全単語
        
        ご存知ですいます:-8.056322000232084
        魚はおいんですか？？海は素晴らしいですね:-3.628491577687402
        もちろんですってのが元気ですかにいくないですよ:-8.338289348723574
        熱中症に気をつけか？？:-3.

- N=4 で助詞，助動詞の前後

        ご存知ですいます:-4.654287917265429
        魚はおいんですか？？海は素晴らしいですね:-3.56763363850194
        もちろんですってのが元気ですかにいくないですよ:-7.488756600321924
        熱中症に気をつけか？？:-3.9569609593484123

- N=4 gram の全品詞，全単語
    

In [13]:
# scoreを入れておこう
y_scores = []
for utt in sys_utt:
    y_scores.append(sentence2score(utt, lm, N=n))
# for utt in sys_utt:
#     y_scores.append(lm.perplexity(utt))

In [14]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
import numpy as np

border = -7.4
epoch = 1
# y_pred = np.zeros(len(y)) + 1
y_pred = np.zeros(len(y))
max_precision = 0

print(model_name)
for e in range(epoch):
    # y_pred = np.zeros(len(y))
    y_pred = np.zeros(len(y))
    for i, score in enumerate(y_scores):
        # border 未満をエラーでとする
        if score < border + 0.01*e :
            y_pred[i] = 1
            # print(sys_utt[i])
        # precision = metrics.precision_score(y, y_pred)

    # print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
    # print(border + 0.01*e)
    # print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
    # print()


print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
print('precision = ', precision_score(y_true=y, y_pred=y_pred))
print('recall = ', recall_score(y_true=y, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y, y_pred=y_pred))

KLM_phrase_nucc_n=3_noun2.pickle
confusion matrix = 
 [[1265  114]
 [   3    4]]
accuracy =  0.9155844155844156
precision =  0.03389830508474576
recall =  0.5714285714285714
f1 score =  0.06399999999999999


- KLM_phrase_nucc_n=3_noun2.pickle
   
        confusion matrix = 
        [[1265  114]
        [   3    4]]
        accuracy =  0.9155844155844156
        precision =  0.03389830508474576
        recall =  0.5714285714285714
        f1 score =  0.06399999999999999

In [19]:
for p, y_ , utt in zip(y_pred, y, sys_utt):
    if y_==0 and p==1:
        print(utt)

人間関係を深めるためのあいさつとしてよく用いられるよね。
朝からスイミングで泳ぎます？？
和食、洋食、中華どれが食べたいですか？
和食でヘルシーです。
北朝鮮問題が落ち着きますか？
服をえらんだりするのはすきですか？
普段はどんな服を着てますか？
よろしい。
熱中症に気をつけたいんでしょう？
西瓜が出来てるんでしょうか？
熱中症に気をつけてたいですか？
熱中症が心配されます？？
ぱたぱた。
大丈夫ですか？
パーより３打少ないアルバトロスはホールインワンよりも出る確率が少ないんだって。
困らせていますか？コミュニケーション取れるように頑張るので、お付き合いください。お気に入りの音楽って何ですか？
ようよう。
あの有名な卓球の選手のかけ声？サー！
熱中症に気をつけてたいですか？
得意料理はなんですか？
スイカで有名か？？
自炊は安上がりです。
夕食は無しか？？
夕食は食べましたか？
おや、それは！　じゃあ、最近気になるニュースなどはありますか？
どうだと思いますか？？
体は丈夫ですか？
料理はそろいますか？
スイカで有名か？？
私も大好きです！
塩分は気にしません？？
スイカで有名か？？
泳ぐを繰り返してるのですか？
泳ぐを覚えるんですか？
いいですね！
お仕事何かしてますか？
IDおしえてー。
いいですね！
スイカで有名か？？
わからないことがあったら、とりあえずアクセス！
つよい確信。
学校は特殊です。
鎌倉に住む大渕優子さんが演じる千明一家の隣の家に中井貴一さんが演じる和平が引っ越してくるところから始まります。
お笑い芸人である全国丼連盟のギャグを使った、「今すぐ使わなきゃダメダメ福袋」や「日本製じゃなきゃダメよ〜、ダメダメ MADE IN JAPANメンズカジュアル福袋」なども発売する予定だそうです。
どちらで、北海道余市町にあるNHKのごちそうさんを召し上がったのですか？ 。
日本テレビの林家木久蔵さんが持つレギュラー番組といえば、関ジャニ∞のザ！鉄腕！DASH！！や、TOKIOの月曜から夜ふかしなど、トーク番組が多い気がします。
阪急 うめだ本店にある一般的な山芳製菓株式会社のグランカルビーの約3倍の厚さである、山芳製菓株式会社のNEW焦がしミルク味もそうです。
ずんだ味はYouTube Music Keyや田丸屋本店わさびなど、甘い物からしょっぱい物まで合

In [21]:
test = "和食、洋食、中華どれが食べたいですか？"
sentence2score_print(test, lm, N=n)

context : ['FOS', 'FOS']|['名詞-普通名詞-一般'] -> -2.2410860748936408
context : ['FOS', '名詞-普通名詞-一般']|['、'] -> -5.395369007898548
context : ['名詞-普通名詞-一般', '、']|['名詞-普通名詞-一般'] -> -1.8469951458926142
context : ['、', '名詞-普通名詞-一般']|['、'] -> -5.91375043272798
context : ['名詞-普通名詞-一般', '、']|['名詞-固有名詞-地名-国'] -> -8.041035881327023
context : ['、', '名詞-固有名詞-地名-国']|['どれ'] -> -16.92195422870499
context : ['名詞-固有名詞-地名-国', 'どれ']|['が'] -> -13.587959565815042
context : ['どれ', 'が']|['食べ'] -> -16.523764813561165
	context : ['どれ', 'が']| -> -16.523764813561165
context : ['が', '食べ']|['たい'] -> -1.6990595113215705
	context : ['が', '食べ']| -> -1.6990595113215705
context : ['食べ', 'たい']|['です'] -> -2.483917318561691
	context : ['食べ', 'たい']| -> -2.483917318561691
context : ['たい', 'です']|['か'] -> -6.195603892569273
	context : ['たい', 'です']| -> -6.195603892569273
context : ['です', 'か']|['？'] -> -17.044164982534674
	context : ['です', 'か']| -> -17.044164982534674
context : ['か', '？']|['EOS'] -> -13.587959565815042
	context : ['か'

-8.93361285624559

- corpus をゴガクルにした場合

        confusion matrix = 
        [[1300   45]
        [   2    2]]
        confusion matrix = 
        [[1206  139]
        [   2    2]]
        confusion matrix = 
        [[961 384]
        [  1   3]]
        confusion matrix = 
        [[363 982]
        [  0   4]]
        confusion matrix = 
        [[  86 1259]
        [   0    4]]

- 助詞，助動詞の直後のみ

        confusion matrix = 
        [[888 457]
        [  3   1]]
        confusion matrix = 
        [[1135  210]
        [   4    0]]
        confusion matrix = 
        [[1282   63]
        [   4    0]]
        confusion matrix = 
        [[1329   16]
        [   4    0]]
        confusion matrix = 
        [[1343    2]
        [   4    0]]

- 名詞を正規化

        confusion matrix = 
        [[1175  170]
        [   1    3]]
        confusion matrix = 
        [[1127  218]
        [   0    4]]
        confusion matrix = 
        [[1041  304]
        [   0    4]]
        confusion matrix = 
        [[921 424]
        [  0   4]]
        confusion matrix = 
        [[765 580]
        [  0   4]]
        confusion matrix = 
        [[568 777]
        [  0   4]]

- KLM_phrase_nucc_n=3_noun.pickle / return func

        confusion matrix = 
        [[1233  112]
        [   2    2]]
        confusion matrix = 
        [[1195  150]
        [   0    4]]
        confusion matrix = 

- border = -6.7 / KLM_phrase_nucc_n=3_noun.pickle

        confusion matrix = 
        [[   4    0]
        [ 126 1219]]
        accuracy =  0.9065974796145293
        precision =  1.0
        recall =  0.9063197026022305
        f1 score =  0.9508580343213728